In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Generate simple sequential data (e.g., stock prices)
X = np.array([[i, i+1, i+2] for i in range(100)])
y = np.array([i+3 for i in range(100)])

# Reshape X for RNN (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build the RNN model
model = Sequential([
    SimpleRNN(10, activation="tanh", input_shape=(3, 1)),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X, y, epochs=50, verbose=1)

# Make predictions
y_pred = model.predict(X[:5])
print("Predictions:", y_pred.flatten())
